<a href="#1-Definition" style="margin-left: 0px;">1 Definition</a>   
<a href="#2-Tree-depth" style="margin-left: 0px;">2 Tree depth</a>  
<a href="#3-Implementation" style="margin-left: 0px;">3 Implementation</a>  

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

### 1 Definition

Random Forest (Случайный лес) — это ансамблевый алгоритм машинного обучения, который строит множество деревьев решений (обычно десятки или сотни) и объединяет их предсказания для повышения точности и устойчивости к переобучению.

**Ансамбль деревьев**

1) Создаётся множество деревьев решений (например, 100), каждое обучается на слегка разных данных.

Случайность на двух уровнях

A) Bagging (Bootstrap Aggregating): Каждое дерево обучается на случайной подвыборке данных (с повторениями).

Б) Случайный выбор признаков: При каждом разбиении узла рассматривается не все признаки, а только их случайное подмножество (например, √n признаков для классификации).

2) Голосование или усреднение

Для классификации: итоговый ответ — класс, выбранный большинством деревьев.

Для регрессии: среднее значение предсказаний всех деревьев.

### 2 Tree depth  
Глубина деревьев  

**1. Random Forest (Случайный лес)**

Типичная глубина деревьев:
Обычно глубокие (почти до полного разделения данных) или с ограничением max_depth (например, 10–30 уровней).

Почему так?
Bagging: Каждое дерево обучается независимо на своей подвыборке данных.

**Цель:** Максимально точное предсказание в рамках своего набора данных (переобучение отдельных деревьев не страшно, так как итоговый результат усредняется).

Случайность: Разные деревья переобучаются на разных аспектах данных, и их голосование компенсирует ошибки.

**2. Бустинг (Gradient Boosting, XGBoost, LightGBM)**

Типичная глубина деревьев:
Обычно мелкие (например, max_depth=3–8), часто это "пни" (stumps) или деревья с 3–6 уровнями.

Почему так?
Последовательное обучение: Каждое новое дерево исправляет ошибки предыдущих (а не обучается независимо).

Ансамбль слабых моделей: Бустинг специально использует недообученные (high bias, low variance) деревья, которые лишь немного улучшают общий результат.

Переобучение: Глубокие деревья в бустинге быстро приводят к переобучению, так как каждое последующее дерево "гиперфокусируется" на ошибках.

### 3 Implementation

#### Classification

| №  | Название колонки | Тип данных       | Описание |
|----|------------------|------------------|----------|
| 1  | `PassengerId`    | `int`            | Уникальный ID пассажира |
| 2  | `Survived`       | `int` (0/1)      | Выжил (1) или нет (0) — **таргет для предсказания** |
| 3  | `Pclass`         | `int` (1-3)      | Класс билета:<br>• 1 = 1-й класс<br>• 2 = 2-й класс<br>• 3 = 3-й класс |
| 4  | `Name`           | `string`         | Имя пассажира (включая титул, например, "Mr.", "Miss") |
| 5  | `Sex`            | `string`         | Пол (`"male"` или `"female"`) |
| 6  | `Age`            | `float`          | Возраст (есть пропуски — `NaN`) |
| 7  | `SibSp`          | `int`            | Количество siblings (братья/сёстры) + spouses (супруги) на борту |
| 8  | `Parch`          | `int`            | Количество parents (родители) + children (дети) на борту |
| 9  | `Ticket`         | `string`         | Номер билета (может содержать буквы и цифры) |
| 10 | `Fare`           | `float`          | Стоимость билета |
| 11 | `Cabin`          | `string`         | Номер каюты (много пропусков — `NaN`) |
| 12 | `Embarked`       | `string`         | Порт посадки:<br>• `C` = Cherbourg<br>• `Q` = Queenstown<br>• `S` = Southampton |

In [2]:
data = pd.read_csv(r"../00 Data/titanic.csv")

print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Замена пропусков в Age медианным значением
data['Age'] = data['Age'].fillna(data['Age'].median())

# Замена пропусков в Embarked модой
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Семейный размер
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# Одиночка (1, если FamilySize = 1)
data['IsAlone'] = (data['FamilySize'] == 1).astype(int)

# Encoder
encoder = TargetEncoder(cols=['Sex', 'Embarked', 'FamilySize'])

data[['Sex', 'Embarked', 'FamilySize']] = encoder.fit_transform(
    data[['Sex', 'Embarked', 'FamilySize']], 
    data['Survived']  # Целевая переменная
)

In [4]:
# StandardScaler для всех признаков
features = data.columns.drop('Survived')
data[features] = StandardScaler().fit_transform(data[features])

In [5]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,0,0.827377,-0.737695,-0.565736,0.432793,-0.473674,-0.502445,-0.539973,1.290322,-1.231645
1,1,-1.566107,1.355574,0.663861,0.432793,-0.473674,0.786845,2.044556,1.290322,-1.231645
2,1,0.827377,1.355574,-0.258337,-0.474545,-0.473674,-0.488854,-0.539973,-0.687981,0.811922
3,1,-1.566107,1.355574,0.433312,0.432793,-0.473674,0.420730,-0.539973,1.290322,-1.231645
4,0,0.827377,-0.737695,0.433312,-0.474545,-0.473674,-0.486337,-0.539973,-0.687981,0.811922


In [6]:
class My_RandomForestClassifier:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=3, min_samples_split=20, bootstrap=True, random_state=42):

        self.n_estimators = n_estimators # количество деревьев
        self.max_features = max_features # Число признаков, которые следует учитывать при поиске наилучшего разбиения
        self.max_depth = max_depth
        self.random_state = random_state
        self.min_samples_split = min_samples_split
        self.bootstrap = bootstrap
        self.trees = []
        self.features_indices = []

    def _bootstrap_sample(self, X, y): # генерируем подвыборки по индексам такого же размера как X, но некоторые элементы
        # будут повторяться, а некоторых не будет
        n_samples = X.shape[0]
        rng = np.random.RandomState(self.random_state)
        indices = rng.choice(n_samples, size=n_samples, replace=True) # replace=True - повторно используются индексы
        return X[indices], y[indices]

    def _get_feature_subset(self, n_features):
        if self.max_features == 'sqrt':
            n_subset = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            n_subset = int(np.log2(n_features))
        elif isinstance(self.max_features, int):
            n_subset = self.max_features
        else:
            raise ValueError("Invalid value for max_features")
        
        rng = np.random.RandomState(self.random_state)
        return rng.choice(n_features, size=n_subset, replace=False)

    def fit(self, X, y):
        self.trees = []
        self.features_indices = []
        
        for _ in range(self.n_estimators): # для каждого дерева
            
            # формируем выборку
            if self.bootstrap == True:
                X_sample, y_sample = self._bootstrap_sample(X, y)
            else:
                X_sample, y_sample = X, y

            # формируем подмножество признаков
            feature_subset = self._get_feature_subset(X.shape[1])
            self.features_indices.append(feature_subset)

            # обучаем модель
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample[:, feature_subset], y_sample)
            self.trees.append(tree)

    def predict(self, X):
        # Создаем пустой список для предсказаний каждого дерева
        predictions = []

        # Проходим по всем деревьям и их признакам
        for i in range(len(self.trees)):
            tree = self.trees[i]
            features = self.features_indices[i] # получаем индексы признаков чтобы сформировать X

            # Получаем предсказания для ОДНОГО ТЕКУЩЕГО дерева
            tree_predictions = tree.predict(X[:, features])

            # Добавляем предсказания дерева в общий список
            predictions.append(tree_predictions)

        # Преобразуем список в numpy массив
        predictions = np.array(predictions)

        # Для каждого примера выбираем наиболее частое предсказание
        final_predictions = []
        for i in range(predictions.shape[1]):  # для каждого примера
            most_common = Counter(predictions[:, i]).most_common(1)  # получаем наиболее частое предсказание
            final_predictions.append(most_common[0][0])  # добавляем его в итоговый список

        return np.array(final_predictions)
                        
    def predict_proba(self, X):
        # Инициализируем массив для вероятностей
        proba = np.zeros((X.shape[0], len(self.trees[0].classes_)))

        # Проходим по всем деревьям и их признакам
        for i in range(len(self.trees)):
            tree = self.trees[i]
            features = self.features_indices[i]

            # Получаем вероятности для текущего дерева
            tree_proba = tree.predict_proba(X[:, features])

            # Добавляем вероятности в общий массив (просто поэлементно складываем вероятности)
            proba += tree_proba

        # Возвращаем средние вероятности по всем деревьям
        return proba / len(self.trees) # делим на количество деревьев чтобы получить среднее

In [7]:
%%time

# Обучаем модель дерева решений
max_features = data.iloc[:,1:].values.shape[1]

my_rf = My_RandomForestClassifier(n_estimators=5, max_depth=5,max_features=max_features, random_state=42)
my_rf.fit(data.iloc[:,1:].values, data.iloc[:,0].values)

CPU times: total: 15.6 ms
Wall time: 25 ms


In [8]:
%%time
my_y_proba = my_rf.predict_proba(data.iloc[:,1:].values)[:,1]

CPU times: total: 0 ns
Wall time: 3 ms


In [9]:
%%time

# Train Random Forest model
max_features = data.iloc[:, 1:].shape[1]

rf = RandomForestClassifier(
    n_estimators=5,
    max_depth=5,
    max_features=max_features,
    random_state=42
)

rf.fit(data.iloc[:, 1:], data.iloc[:, 0])

CPU times: total: 31.2 ms
Wall time: 30.1 ms


RandomForestClassifier(max_depth=5, max_features=9, n_estimators=5,
                       random_state=42)

In [10]:
%%time
y_proba = rf.predict_proba(data.iloc[:,1:])[:,1]

CPU times: total: 0 ns
Wall time: 8.04 ms


In [11]:
# Вычисление AUC-ROC
AUC_my = roc_auc_score(data['Survived'].values, my_y_proba)
AUC_sclearn = roc_auc_score(data['Survived'].values, y_proba)

print(f"AUC-ROC (Custom Model): {AUC_my:.4f}")
print(f"AUC-ROC (Custom Model): {AUC_sclearn:.4f}")

AUC-ROC (Custom Model): 0.8770
AUC-ROC (Custom Model): 0.9183


#### Результат схожий

#### Регрессия

In [12]:
data = pd.read_csv(r"../00 Data/real estate.csv")

In [13]:
print(data.shape)
data.head()

(49352, 24)


,bathrooms,bedrooms,interest_level,price,Elevator,CatsAllowed,HardwoodFloors,DogsAllowed,Doorman,Dishwasher,...,LaundryinUnit,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
0,1.0,1,1,2400,0,1,1,1,0,1,...,0,0,0,1,0,0,0,0,0,0
1,1.0,2,0,3800,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2,1,3495,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
3,1.5,3,1,3000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,2795,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = data.drop(['price'],axis=1).to_numpy()
y = data['price'].to_numpy()

In [15]:
class My_RandomForestRegressor:
    
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=3, 
                 min_samples_split=20, bootstrap=True, random_state=42):

        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.bootstrap = bootstrap
        self.random_state = random_state
        self.trees = []
        self.features_indices = []

    # Создает бутстрэп-выборку
    def _bootstrap_sample(self, X, y):
        
        n_samples = X.shape[0]
        rng = np.random.RandomState(self.random_state)
        indices = rng.choice(n_samples, size=n_samples, replace=True)
        return X[indices], y[indices]

    # Выбирает случайное подмножество признаков
    def _get_feature_subset(self, n_features):
        if self.max_features == 'sqrt':
            n_subset = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            n_subset = int(np.log2(n_features))
        elif isinstance(self.max_features, int):
            n_subset = min(self.max_features, n_features)
        else:
            raise ValueError("Недопустимое значение для max_features")
        
        rng = np.random.RandomState(self.random_state)
        return rng.choice(n_features, size=n_subset, replace=False)

    # Обучение случайного леса
    def fit(self, X, y):

        self.trees = []
        self.features_indices = []
        
        for _ in range(self.n_estimators):
            # 1. Создаем бутстрэп-выборку
            if self.bootstrap:
                X_sample, y_sample = self._bootstrap_sample(X, y)
            else:
                X_sample, y_sample = X, y

            # 2. Выбираем случайные признаки
            feature_subset = self._get_feature_subset(X.shape[1])
            self.features_indices.append(feature_subset)

            # 3. Обучаем дерево регрессии
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                random_state=self.random_state
            )
            tree.fit(X_sample[:, feature_subset], y_sample)
            self.trees.append(tree)

    # Предсказание (усреднение прогнозов всех деревьев)
    def predict(self, X):
        
        predictions = np.zeros(X.shape[0])
        
        for i, tree in enumerate(self.trees):
            predictions += tree.predict(X[:, self.features_indices[i]])
        
        return predictions / self.n_estimators

    # Вычисление среднеквадратичной ошибки
    def get_mse(self, X, y_true):
        
        y_pred = self.predict(X)
        return np.mean((y_true - y_pred)**2)

In [16]:
%%time
# 1. Создание модели дерева решений
my_model_rf = My_RandomForestRegressor(max_depth=5)

# 2. Обучение модели
my_model_rf.fit(X,y)

CPU times: total: 3.12 s
Wall time: 3.13 s


In [17]:
%%time
# 3. Получение предсказаний
my_y_pred = my_model_rf.predict(X)

CPU times: total: 453 ms
Wall time: 450 ms


In [18]:
%%time
# 1. Создание модели дерева решений
model_rf = RandomForestRegressor(max_depth=5)

# 2. Обучение модели
model_rf.fit(X,y)

CPU times: total: 4.3 s
Wall time: 4.29 s


RandomForestRegressor(max_depth=5)

In [19]:
# 3. Получение предсказаний
y_pred = model_rf.predict(X)

In [20]:
my_mse = mean_squared_error(y, my_y_pred)
mse = mean_squared_error(y, y_pred)

print(f'Mean Squared Error: {my_mse}')
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 484120690.1741019
Mean Squared Error: 482429128.2048073


#### Результат схожий